In [2]:
import os
import cv2
import math
import random
import numpy as np
from skimage.util import random_noise
from PIL import Image, ImageDraw

## Costanti

In [3]:
# PATH_DATASET_LAVORATO = './Dati Sensori/Dataset Sensori Lavorato'
# PATH_DATASET_NOISE = './Dati Sensori/Dataset Sensori Noise'
# PATH_DATASET_ART_RIF = './Dati Sensori/Dataset Sensori Artefatti Riflessione'
# PATH_DATASET_SPECKLE = './Dati Sensori/Dataset Sensori Speckle'
# PATH_DATASET_BLUR = './Dati Sensori/Dataset Sensori Motion Blur'
# PATH_DATASET_LINEE = './Dati Sensori/Dataset Sensori Linee'

In [21]:
PATH_DATASET_LAVORATO = './Nuovi Dati/TemplateGenerati'
PATH_DATASET_SPECKLE = './Nuovi Dati/DatasetDistorti/DatasetSpeckle'
PATH_DATASET_BLUR = './Nuovi Dati/DatasetDistorti/DatasetBlur'
PATH_DATASET_BLACK_STAINS = './Nuovi Dati/DatasetDistorti/DatasetBlackStains'
PATH_DATASET_LINEE = './Nuovi Dati/DatasetDistorti/DatasetLinee'

## Configurazione

In [18]:
def genera_immagini_distorte(dir_img_orig, dir_img_dist, callback_distorsione, **kwargs):
    for root, dirs, files in os.walk(dir_img_orig):
        for file in files:
            if not file.endswith("immagine1.jpg"): continue
            print(file)
            path_img_orig = os.path.join(root, file)
            path_img_dist = path_img_orig.replace(dir_img_orig, dir_img_dist)
            callback_distorsione(path_img_orig, path_img_dist, **kwargs)
            

## Rumore gaussiano

## Artefatti di riflessione
Vengono modificati i valori dei pixel per riflettere l'immagine rispetto all'asse di simmetria.
L'immagine riflessa viene sovrapposta all'immagine originale per ottenere un effetto di fusione tra le due immagini.  
vd. https://onlinelibrary.wiley.com/doi/abs/10.7863/jum.1986.5.4.227

In [ ]:
def applica_artefatti_riflessione(path_img_orig, path_img_dist, peso_img_rifl=0.5):
        print(f"** Applico riflessione: P{peso_img_rifl} a {path_img_orig} ...")
        img = cv2.imread(path_img_orig)
        img_riflessa = img.copy()
        h, w, _ = img.shape
        centro_x = int(w/2)
        for y in range(h):
            for x in range(centro_x, w):
                x_rifl = w - x - 1  # Posizione x del pixel riflesso
                img_riflessa[y, x] = img[y, x_rifl]
        blended_img = cv2.addWeighted(img, 1-peso_img_rifl, img_riflessa, peso_img_rifl, 0)
        os.makedirs(os.path.dirname(path_img_dist), exist_ok=True)
        cv2.imwrite(path_img_dist, blended_img)

In [ ]:
%%time
nPesi = 7
pesi_img_riflessa = [round(0.1 * i, 2) for i in range(nPesi)]

for i, peso_rifl in enumerate(pesi_img_riflessa):
    dir_img_dist = f"{PATH_DATASET_ART_RIF}-{i}"
    genera_immagini_distorte(PATH_DATASET_LAVORATO, f"{PATH_DATASET_ART_RIF}-{i}", applica_artefatti_riflessione, peso_img_rifl=peso_rifl)

## Speckle

In [19]:
def applica_speckle(path_img_orig, path_img_dist, speckle_mean=0, speckle_std=0.1):
    print(f"** Applico speckle: M{speckle_mean}-S{speckle_std} a {path_img_orig} ...")
    img = cv2.imread(path_img_orig, 0)
    speckle_noise = np.random.normal(speckle_mean, speckle_std, img.shape)
    img_rumore = img + img * speckle_noise
    img_rumore = cv2.normalize(img_rumore, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    os.makedirs(os.path.dirname(path_img_dist), exist_ok=True)
    cv2.imwrite(path_img_dist, img_rumore)

In [20]:
%%time
nStd = 5
nMean = 3

speckle_mean_list = [round(0.1 * i, 2) for i in range(nMean)]  
speckle_std_list = [round(0.1 * i, 2) for i in range(nStd)]

for i, speckle_mean in enumerate(speckle_mean_list):
    for j, speckle_std in enumerate(speckle_std_list):
        dir_img_dist = f"{PATH_DATASET_SPECKLE}-m{i}-s{j}"
        genera_immagini_distorte(PATH_DATASET_LAVORATO, dir_img_dist, applica_speckle, speckle_mean=speckle_mean, speckle_std=speckle_std)

immagine1.jpg
** Applico speckle: M0.0-S0.0 a ./Nuovi Dati/TemplateGenerati/CasaleRocco_005/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico speckle: M0.0-S0.0 a ./Nuovi Dati/TemplateGenerati/ContinoloGiulia_001/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico speckle: M0.0-S0.0 a ./Nuovi Dati/TemplateGenerati/CiprianoInnella_000/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico speckle: M0.0-S0.0 a ./Nuovi Dati/TemplateGenerati/TummilloBrigida_011/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico speckle: M0.0-S0.0 a ./Nuovi Dati/TemplateGenerati/MoccardiFrancesco_002/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico speckle: M0.0-S0.0 a ./Nuovi Dati/TemplateGenerati/CasaleRocco_002/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico speckle: M0.0-S0.0 a ./Nuovi Dati/TemplateGenerati/TeodosioAntonio_012/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico speckle: M0.0-S0.0 a ./Nuovi Dati/TemplateGenerati/AgatielloRoberto_012/Immagini/immagine1.jpg ...
immagine1.jpg
** Appl

## Linee Orizzontali

In [71]:
def applica_linee(path_img_orig, path_img_dist, line_y_positions, sizes):
    print(f"** Applico Linee Orizzontali a {path_img_orig} ...")    
    img = Image.open(path_img_orig)

    draw = ImageDraw.Draw(img)
    
    for i, y in enumerate(line_y_positions):        
        starting_x = 0
        ending_x = img.width
        line_width = sizes[i]
        actual_x = 0
        dash_size = 2
        while actual_x + dash_size < ending_x:
            start_point = (actual_x, y)
            end_point = (actual_x + dash_size, y)
            draw.line([start_point, end_point], width=sizes[i])
            actual_x += 4*dash_size
        
    os.makedirs(os.path.dirname(path_img_dist), exist_ok=True)
    img.save(path_img_dist)

In [ ]:
%%time
n_max_linee = 4

linee = [[0]]
dim_linee = [[0]]
for n_linee in range(1, n_max_linee + 1):
    linea = [0 for i in range(n_linee)]
    size = [0 for i in range(n_linee)]
    for j in range(n_linee):
        linea[j] = random.randrange(1, 500)
        size[j] = random.randrange(4, 8)
    linee.append(linea)
    dim_linee.append(size)
    

for i, linea in enumerate(linee):
        dir_img_dist = f"{PATH_DATASET_LINEE}-y{i}"
        genera_immagini_distorte(PATH_DATASET_LAVORATO, dir_img_dist, applica_linee, line_y_positions=linee[i], sizes=dim_linee[i])

## Motion Blur

In [74]:
def applica_motion_blur(path_img_orig, path_img_dist, size=0):
    print(f"** Applico Motion Blur: S{size} a {path_img_orig} ...")    
    img = cv2.imread(path_img_orig, 0)

    kernel_motion_blur = np.zeros((size, size))
    kernel_motion_blur[:, int((size-1)/2)] = np.ones(size)
    kernel_motion_blur = kernel_motion_blur / size

    img_blurred = cv2.filter2D(img, -1, kernel_motion_blur)
    
    img_blurred = cv2.normalize(img_blurred, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    os.makedirs(os.path.dirname(path_img_dist), exist_ok=True)
    cv2.imwrite(path_img_dist, img_blurred)

In [75]:
%%time
n_fattori = 4

#size_list = [1] + [3*((2*i)+1) for i in range(n_fattori)]
size_list = [1,2,3,5,8]

for i, size in enumerate(size_list):
        dir_img_dist = f"{PATH_DATASET_BLUR}-s{i}"
        path_dataset_linee = f'{PATH_DATASET_LINEE}-y{i}'
        genera_immagini_distorte(path_dataset_linee, dir_img_dist, applica_motion_blur, size=size)

immagine1.jpg
** Applico Motion Blur: S1 a ./Nuovi Dati/DatasetDistorti/DatasetLinee-y0/CasaleRocco_005/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico Motion Blur: S1 a ./Nuovi Dati/DatasetDistorti/DatasetLinee-y0/ContinoloGiulia_001/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico Motion Blur: S1 a ./Nuovi Dati/DatasetDistorti/DatasetLinee-y0/CiprianoInnella_000/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico Motion Blur: S1 a ./Nuovi Dati/DatasetDistorti/DatasetLinee-y0/TummilloBrigida_011/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico Motion Blur: S1 a ./Nuovi Dati/DatasetDistorti/DatasetLinee-y0/MoccardiFrancesco_002/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico Motion Blur: S1 a ./Nuovi Dati/DatasetDistorti/DatasetLinee-y0/CasaleRocco_002/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico Motion Blur: S1 a ./Nuovi Dati/DatasetDistorti/DatasetLinee-y0/TeodosioAntonio_012/Immagini/immagine1.jpg ...
immagine1.jpg
** Applico Motion Blur: S1 a ./Nuovi Dati/Datas